References: StackOverflow, Dr. Alvarado starter code files and provided Python scripts,https://cs.stanford.edu/people/eroberts/courses/soco/projects/1999-00/information-theory/redundancy_5.html

In [2]:
## Standard imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from IPython.core.display import HTML

## Import modules

sys.path.insert(0, '/Users/nkeeley/DS5001-2022-01/lib')
from textimporter import *
from langmod import *

## Set path for text

sys.path.insert(1, '/Users/nkeeley/DS5001-2022-01')
text_file="/Users/nkeeley/DS5001-2022-01/data/gutenberg/pg42324.txt"

In [3]:
data_home = "../data"

In [4]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

# Import file into a dataframe

In [5]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [6]:
LINES.sample(20)

,line_str
line_num,
1048,"the antique glory of Italy,--one among the _sc..."
3010,"tempted, when all was at peace around me, and ..."
4089,"court. He made, at that moment, a solemn vow t..."
4125,"""Safie related, that her mother was a Christia..."
5798,"filled me with such agreeable sensations, that..."
3415,down and wept.
6314,"impatient thirst for sympathy, and was silent ..."
4526,"""I paused. This, I thought, was the moment of ..."
1418,"""Have you,"" he said, ""really spent your time i..."


# Extract Title 

In [7]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [8]:
print(title)

Frankenstein, by Mary W. Shelley


# Clip Cruft

In [9]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT", 
    r"\n*THE END."
]
#r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"

In [10]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [11]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] + 1

In [12]:
line_a, line_b

(19, 7656)

In [13]:
## Good. This captures everything to THE END.

LINES = LINES.loc[line_a : line_b]
LINES.tail()

,line_str
line_num,
7652,lost in darkness and distance.
7653,
7654,
7655,THE END.
7656,


In [14]:
LINES.iloc[300:350]

,line_str
line_num,
319,desire of imitation. Two other friends (a tale...
320,whom would be far more acceptable to the publi...
321,ever hope to produce) and myself agreed to wri...
322,on some supernatural occurrence.
323,
324,"The weather, however, suddenly became serene; ..."
325,"on a journey among the Alps, and lost, in the ..."
326,"they present, all memory of their ghostly visi..."
327,the only one which has been completed.


In [15]:
LINES.sample(10)

,line_str
line_num,
2366,"""ALPHONSE FRANKENSTEIN."
1107,"On the birth of a second son, my junior by sev..."
6631,"of objects, a thousand fears arose in my mind...."
5645,in vain endeavour to fly from an impending dan...
4943,"persevere in this exile? You will return, and ..."
7391,"guest, whose illness increased in such a degre..."
7531,"""After the murder of Clerval, I returned to Sw..."
1427,find a disciple of Albertus Magnus and Paracel...
1792,"It was on a dreary night of November, that I b..."


# Chunk by chapter

## Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [16]:
## PREFACE and CHAPTER I, II, etc.

chap_pat = r"(PREFACE*|CHAPTER*|(LETTER\sI*))"

In [18]:
LINES.line_str.str

In [17]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [18]:
LINES.loc[chap_lines] # Use as filter for dataframe

,line_str
line_num,
271,PREFACE.
343,LETTER I.
467,LETTER II.
594,LETTER III.
636,LETTER IV.
918,CHAPTER I.
1085,CHAPTER II.
1299,CHAPTER III.
1555,CHAPTER IV.


## Assign numbers to chapters

In [1]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

NameError: name 'LINES' is not defined

In [20]:
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
271,PREFACE.,1.0
343,LETTER I.,2.0
467,LETTER II.,3.0
594,LETTER III.,4.0
636,LETTER IV.,5.0
918,CHAPTER I.,6.0
1085,CHAPTER II.,7.0
1299,CHAPTER III.,8.0
1555,CHAPTER IV.,9.0


Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [21]:
LINES.sample(10)

,line_str,chap_num
line_num,,
6108,which I discovered from its commencement to be...,NaN
6450,"affectionate. ""I fear, my beloved girl,"" I sai...",NaN
2074,"widow with four children, of whom Justine was ...",NaN
2053,"asks but to see you,--but to be assured that y...",NaN
1792,"It was on a dreary night of November, that I b...",NaN
7533,abhorred myself. But when I discovered that he...,NaN
2543,William; and my tears flowed when I looked upo...,NaN
5425,patriot fell. For a moment my soul was elevate...,NaN
382,"all these conjectures to be false, you cannot ...",NaN


## Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [22]:
LINES.chap_num = LINES.chap_num.ffill()

In [23]:
LINES.sample(10)

,line_str,chap_num
line_num,,
3396,"suppose, the light poured in upon me again. I ...",16.0
6881,"And now my wanderings began, which are to ceas...",29.0
352,forebodings. I arrived here yesterday; and my ...,2.0
1585,there is continual food for discovery and wond...,9.0
1518,"said, that ""these were men to whose indefatiga...",8.0
2391,he could only express his heart-felt sympathy....,12.0
2448,"at Secheron, a village at the distance of half...",12.0
7112,In this manner many appalling hours passed; se...,29.0
3067,who can assure themselves of certain happiness...,14.0


Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [24]:
LINES.head(20)

,line_str,chap_num
line_num,,
19,,NaN
20,,NaN
21,,NaN
22,,NaN
23,"Produced by Greg Weeks, Mary Meehan and the On...",NaN
24,Distributed Proofreading Team at http://www.pg...,NaN
25,,NaN
26,,NaN
27,,NaN


## Clean up

In [25]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [26]:
LINES.tail(20)

,line_str,chap_num
line_num,,
7637,"and feel, thou wouldst not desire against me a...",29
7638,"that which I feel. Blasted as thou wert, my ag...",29
7639,thine; for the bitter sting of remorse will no...,29
7640,wounds until death shall close them for ever.,29
7641,,29
7642,"""But soon,"" he cried, with sad and solemn enth...",29
7643,what I now feel be no longer felt. Soon these ...,29
7644,extinct. I shall ascend my funeral pile triump...,29
7645,agony of the torturing flames. The light of th...,29


## Group lines into chapters

In [27]:
OHCO[:1]

['chap_num']

In [28]:
CHAPS = LINES.groupby(OHCO[:1]).line_str\
    .apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str') # Make big string for each chapter

In [29]:
CHAPS.head(10)

,chap_str
chap_num,
1,\n\nThe event on which this fiction is founded...
2,"\n\n_To Mrs. Saville, England._\n\nSt. Petersb..."
3,"\n\n_To Mrs. Saville, England._\n\nArchangel, ..."
4,"\n\n_To Mrs. Saville, England._\n\nMY DEAR SIS..."
5,"\n\n_To Mrs. Saville, England._\n\nAugust 5th,..."
6,\n\nI am by birth a Genevese; and my family is...
7,\n\nWe were brought up together; there was not...
8,"\n\nWhen I had attained the age of seventeen, ..."
9,"\n\nFrom this day natural philosophy, and part..."


In [30]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [31]:
CHAPS.head(30)

,chap_str
chap_num,
1,"The event on which this fiction is founded, ha..."
2,"_To Mrs. Saville, England._\n\nSt. Petersburgh..."
3,"_To Mrs. Saville, England._\n\nArchangel, 28th..."
4,"_To Mrs. Saville, England._\n\nMY DEAR SISTER,..."
5,"_To Mrs. Saville, England._\n\nAugust 5th, 17-..."
6,I am by birth a Genevese; and my family is one...
7,We were brought up together; there was not qui...
8,"When I had attained the age of seventeen, my p..."
9,"From this day natural philosophy, and particul..."


So, now we have our text grouped by chapters.

# Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [32]:
para_pat = r'\n\n+'

In [33]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [34]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [35]:
PARAS.tail()

para_str
chap_num para_num                                                   
29       79        "Fear not that I shall be the instrument of fu...
         80        "Farewell! I leave you, and in you the last of...
         81        "But soon," he cried, with sad and solemn enth...
         82        He sprung from the cabin-window, as he said th...
         83                                                 THE END.

In [36]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [37]:
PARAS.tail()

para_str
chap_num para_num                                                   
29       79        "Fear not that I shall be the instrument of fu...
         80        "Farewell! I leave you, and in you the last of...
         81        "But soon," he cried, with sad and solemn enth...
         82        He sprung from the cabin-window, as he said th...
         83                                                 THE END.

# Split paragraphs into sentences

In [38]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [39]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [40]:
SENTS.tail(1).sent_str

chap_num  para_num  sent_num
29        83        0           THE END
Name: sent_str, dtype: object

# Split sentences into tokens

In [41]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [42]:
TOKENS.index.names = OHCO[:4]

In [43]:
TOKENS

token_str
chap_num para_num sent_num token_num          
1        0        0        0               The
                           1             event
                           2                on
                           3             which
                           4              this
...                                        ...
29       82       1        11         darkness
                           12              and
                           13         distance
         83       0        0               THE
                           1               END

[76464 rows x 1 columns]

# Extract Vocabulary

In [44]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [45]:
VOCAB

,term_str,n
term_id,,
0,the,4250
1,and,2993
2,i,2859
3,of,2684
4,to,2119
...,...,...
7021,arrow,1
7022,2d,1
7023,speculation,1


# Gathering by Content Object

In [46]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [47]:
gather(4)

token_str
chap_num para_num sent_num token_num          
1        0        0        0               The
                           1             event
                           2                on
                           3             which
                           4              this
...                                        ...
29       82       1        11         darkness
                           12              and
                           13         distance
         83       0        0               THE
                           1               END

[76464 rows x 1 columns]

chap_num  para_num  sent_num
1         0         0           The event on which this fiction is founded has...
                    1           Darwin and some of the physiological writers o...
                    2           I shall not be supposed as according the remot...
                    3           yet in assuming it as the basis of a work of f...
                    4           The event on which the interest of the story d...
                                                      ...                        
29        81        7                                                    Farewell
                    8                                                           "
          82        0           He sprung from the cabin window as he said thi...
                    1           He was soon borne away by the waves and lost i...
          83        0                                                     THE END
Name: sent_str, Length: 5212, dtype: object

## Ngram Creation

In [48]:
## Create list of sentences from TOKEN table

test=gather(3)
test=test.sent_str.reset_index()
test=test.sent_str
test=list(test)
type(test)
S_train=test

## Create list of terms from VOCAB table

test=VOCAB.reset_index()
terms=sorted(list(test.term_str))
V_train=terms

## Create NgramCounter -- assuming n is maximum level of ngram

counter = NgramCounter(S_train, V_train, 3)

## Generate tables

counter.generate()
counter.I

w0     w1     w2
sent_num token_num                     
0        0            <s>    <s>  <UNK>
         1            <s>  <UNK>  event
         2          <UNK>  event     on
         3          event     on  which
         4             on  which   this
...                   ...    ...    ...
5211     0            <s>    <s>  <UNK>
         1            <s>  <UNK>  <UNK>
         2          <UNK>  <UNK>   </s>
         3          <UNK>   </s>    NaN
         4           </s>    NaN    NaN

[91938 rows x 3 columns]

In [49]:
## Query monster

counter.I.query("w2 == 'monster'")

,,w0,w1,w2
sent_num,token_num,,,
979,25,the,miserable,monster
1050,5,behold,this,monster
1081,5,that,the,monster
1095,4,of,the,monster
1789,14,was,the,monster
1919,8,lest,the,monster
2105,1,<s>,<UNK>,monster
2467,12,reality,the,monster
2605,4,then,a,monster


In [50]:
## Train Model

train = NgramCounter(S_train, V_train, 2)
train.generate()
model=NgramLanguageModel(train)
model.k=1
model.apply_smoothing()
NG = model.NG
LM = model.LM
Z1 = model.Z1
Z2 = model.Z2

In [55]:
## Test sentences

S_TEST = """
The monster is on the ice.
Flowers are happy things.
I have never seen the aurora borealis.
He never knew the love of a family.
""".split('\n')[1:-1]

test = NgramCounter(S_TEST, V_train) # Note that we use the training vocab
test.generate()
model.predict(test)
comp_cols = ['len','pp1','pp2','sent_str']
test.S.sort_values('pp1')[comp_cols].sent_str

1                 Flowers are happy things.
0                The monster is on the ice.
2    I have never seen the aurora borealis.
3       He never knew the love of a family.
Name: sent_str, dtype: object

In [52]:
## Question 3

model.LM[1].unstack().n.loc[["he","she"],['said','heard']]

w1,said,heard
w0,,
he,18.0,4.0
she,3.0,3.0


In [53]:
## Question 4

sents = model.generate_text()

01. THEY DIED AWAY BY BRIGHT SUN HAD GROWN PALE RADIANCE AMONG THEM INTO THE <UNK> MOMENT <UNK> <UNK> <UNK> THOUGHT WHAT A SMALL PRICE TO HOPE REVISIT MY PERSECUTOR FOR FALSEHOOD <UNK> GIRL <UNK> SEEK ONE WAND RING THOUGHT THE THREAT OF HUMAN BEINGS FROM ALL THAT HE WAS OBLIGED US INDICATING THE PEASANTS OF USE OF TOWN <UNK> CONTINUED A LICENCE OR THREE MONTHS LONGER SEE YOU ON THIS RETREAT AS THEY OBSERVED MY PRESENT SENSATIONS WHO IT WITH ATTENTION WAS TOO CONNECTED WITH WISHED THAT TASK TO ITS ORB OF THE MONSTER <UNK> LIGHT AS A CELESTIAL EYES AND CLOSED IN THE GRAVES OF LONELY MADDENING RAGE WAS EXPOSED TO MY UNEARTHLY UGLINESS <UNK> COULD NOT BEEN SO LIKE A BROTHER <UNK> SOON AS YOU TO UNITE THEM MORE MIGHT BECOME AN INVISIBLE HAND <UNK> THANK YOU SEE THE FATE MY ENEMIES MY FELLOW CREATURES IN A MURDERER <UNK> <UNK> AND THEIR BLACK MARK TO YOU APPEAR LIKE A FEW PREPARATORY EXPERIMENTS HE ENQUIRED IF YOU MEAN THAT MY CONVALESCENCE <UNK> HAD RETIRED TO WEEP WITH ME BE A MERE PRESENC

In [1]:
## Question 5 - generate ngrams and compute redundancy

model.LM[1]
counter = NgramCounter(S_train, V_train, 3)
counter.generate()
train = NgramCounter(S_train, V_train, 3)
train.generate()
model=NgramLanguageModel(train)
model.k=1
model.apply_smoothing()
NG = model.NG
LM = model.LM
Z1 = model.Z1
Z2 = model.Z2

## Unigram

model_uni=model.LM[0]
model_uni["l_inverse"]=np.log2(1/model_uni.p)
model_uni["summation"]=model_uni.l_inverse * model_uni.p
max_e=(max(model_uni.p)*model_uni.shape[0])
entropy_uni=sum(model_uni.summation)
red_uni=1-(entropy_uni/max_e)
red_uni
#0.9892102479283928

## Bigram

model_uni=model.LM[1]
model_uni["l_inverse"]=np.log2(1/model_uni.p)
model_uni["summation"]=model_uni.l_inverse * model_uni.p
max_e=(max(model_uni.p)*model_uni.shape[0])
entropy_uni=sum(model_uni.summation)
red_uni=1-(entropy_uni/max_e)
red_uni
#0.9901909097489813

## Trigram

model_uni=model.LM[2]
model_uni["l_inverse"]=np.log2(1/model_uni.p)
model_uni["summation"]=model_uni.l_inverse * model_uni.p
max_e=(max(model_uni.p)*model_uni.shape[0])
entropy_uni=sum(model_uni.summation)
red_uni=1-(entropy_uni/max_e)
red_uni
#0.98920943440478

model_uni.shape

NameError: name 'model' is not defined

In [86]:
## Question 5 - redundancy round 2

model.LM[1]
counter = NgramCounter(S_train, V_train, 3)
counter.generate()
train = NgramCounter(S_train, V_train, 3)
train.generate()
model=NgramLanguageModel(train)
model.k=1
model.apply_smoothing()
NG = model.NG
LM = model.LM
Z1 = model.Z1
Z2 = model.Z2

## Unigram

model_uni=model.LM[0]
model_uni["log_inverse"]=np.log2(1/model_uni.mle)
entropy=sum(model_uni.mle*model_uni.log_inverse)
entropy #8.25
V=len(V_train)
Hmax=np.log2(V**1) # 
R=1-entropy/Hmax
R # 0.3539793127793027

## Bigram

model_uni=model.LM[1]
model_uni["log_inverse"]=np.log2(1/model_uni.mle)
entropy=sum(model_uni.mle*model_uni.log_inverse)
entropy #12.8
V=len(V_train)
Hmax=np.log2(V**2) # 12.8
R=1-entropy/Hmax
R # 0.49949967928782757


## Trigram

model_uni=model.LM[2]
model_uni["log_inverse"]=np.log2(1/model_uni.mle)
entropy=sum(model_uni.mle*model_uni.log_inverse)
entropy #14.6
V=len(V_train)
Hmax=np.log2(V**3) # 12.8
R=1-entropy/Hmax
R # 0.6195379807141475




14.585187885301258

## END OF FILE: Non-manual below

In [37]:
## Adjust file path and import modules

sys.path.insert(0, '/Users/nkeeley/DS5001-2022-01/lib')
from textimporter import *
from langmod import *


In [80]:
## Read in raw text

text_file="/Users/nkeeley/DS5001-2022-01/data/gutenberg/pg42324.txt"
#raw=open(text_file, 'r').read()
#raw 
text_file

'/Users/nkeeley/DS5001-2022-01/data/gutenberg/pg42324.txt'

In [95]:
## Create the TextImporter object

#sys.path.insert(0, '/Users/nkeeley/DS5001-2022-01/data/gutenberg')
ohco_pats:[] = [
    ('para', r"\n\n", 'd'),
    ('sent', r"[.?!;:]+", 'd'),
    ('token', r"[\s',-]+", 'd')
]

_ohco_type:{} = {
    'd': '_num',
    'm': '_id'
}   
    
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT", 
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]
ohco1 = TextImporter(src_file=text_file, ohco_pats=ohco_pats, clip_pats=clip_pats)

In [96]:
## Parse to create VOCAB and TOKEN tables


ohco1.import_source()
ohco1.parse_tokens()
#ohco1.extract_vocab()


Importing  /Users/nkeeley/DS5001-2022-01/data/gutenberg/pg42324.txt
Clipping text
Parsing OHCO level 0 para_num by delimitter \n\n


KeyError: 'token_str'

In [76]:
ohco2

NameError: name 'ohco2' is not defined